In [21]:
import sklearn as sk
import numpy as np
import pandas as pd
import csv

In [35]:
#Read in the data as a DataFrame
df = pd.DataFrame.from_csv("/Users/msteiner/GA/OnlineNews_project/DataStore/data/OnlineNewsPopularity.csv",sep=',',header=0,index_col=None)

In [36]:
#limit the number of records for code developement -- remove to run using full data set
df = df[0:1000]


In [37]:
df.columns.values


array(['url', ' timedelta', ' n_tokens_title', ' n_tokens_content',
       ' n_unique_tokens', ' n_non_stop_words',
       ' n_non_stop_unique_tokens', ' num_hrefs', ' num_self_hrefs',
       ' num_imgs', ' num_videos', ' average_token_length',
       ' num_keywords', ' data_channel_is_lifestyle',
       ' data_channel_is_entertainment', ' data_channel_is_bus',
       ' data_channel_is_socmed', ' data_channel_is_tech',
       ' data_channel_is_world', ' kw_min_min', ' kw_max_min',
       ' kw_avg_min', ' kw_min_max', ' kw_max_max', ' kw_avg_max',
       ' kw_min_avg', ' kw_max_avg', ' kw_avg_avg',
       ' self_reference_min_shares', ' self_reference_max_shares',
       ' self_reference_avg_sharess', ' weekday_is_monday',
       ' weekday_is_tuesday', ' weekday_is_wednesday',
       ' weekday_is_thursday', ' weekday_is_friday',
       ' weekday_is_saturday', ' weekday_is_sunday', ' is_weekend',
       ' LDA_00', ' LDA_01', ' LDA_02', ' LDA_03', ' LDA_04',
       ' global_subjectivity',

In [38]:
#Take a subset of features based on Ren, Yang et al. 
# A feature selection process still must be done from scrtach -- ITEM TO BE DONE --
df_ren = df[[' kw_avg_avg',' LDA_02',' data_channel_is_world',' is_weekend',' data_channel_is_socmed',' weekday_is_saturday',
    ' LDA_04', ' data_channel_is_entertainment',' data_channel_is_tech',' kw_max_avg',' weekday_is_sunday',
    ' LDA_00', ' num_hrefs',' global_subjectivity',' kw_avg_min',' title_sentiment_polarity',' rate_negative_words',
   ' kw_min_avg',' title_subjectivity',' LDA_01']]

In [39]:
#the features
features_ren = df_ren

In [40]:
#get the dependent variable 
target_ren = df[' shares']

In [41]:
#map continuous response variable to integral values
#shares >= 1400 map to 1
#shares < 1400 map to 0 

target_ren_mapped = np.zeros(len(target_ren))
counter = 0
for ii in target_ren:
    if ii >= 1400:
        target_ren_mapped[counter] = 1
    else:
        target_ren_mapped[counter] = 0
    counter += 1

In [45]:
# Standardize/normalize the features
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(df_ren)
features_ren = scaler.transform(df_ren)

In [31]:
#normalize the data different method
#columns_to_normalize = [' kw_avg_avg',' LDA_02',' data_channel_is_world',' is_weekend',' data_channel_is_socmed',' weekday_is_saturday',
#    ' LDA_04', ' data_channel_is_entertainment',' data_channel_is_tech',' kw_max_avg',' weekday_is_sunday',
#    ' LDA_00', ' num_hrefs',' global_subjectivity',' kw_avg_min',' title_sentiment_polarity',' rate_negative_words',
#   ' kw_min_avg',' title_subjectivity',' LDA_01']
#features_ren[columns_to_normalize] = features_ren[columns_to_normalize].apply(lambda x: (x - x.mean())/x.std())

# SUPPORT VECTOR MACHINE
### Linear Kernel

In [48]:
# import the SVM class
from sklearn.svm import SVC
svc_1 = SVC(kernel = 'linear')

In [49]:
#split data into test and training
from sklearn.cross_validation import train_test_split
features_ren_train, features_ren_test, target_ren_train, target_ren_test = train_test_split(
features_ren, target_ren_mapped, test_size=0.25, random_state=0)

In [61]:
#employ K-fold cross validation 
from sklearn.cross_validation import cross_val_score, KFold
from scipy.stats import sem

def evaluate_cross_validation(clf, X, y, K):
    cv = KFold(len(y),K, shuffle=True, random_state=0)
    scores = cross_val_score(clf, X, y, cv=cv)
    print scores
    print(("Mean score: {0:.3f} (+/-{1:.3f})").format(np.mean(scores), sem(scores)))

In [62]:
#show results of cross validation
evaluate_cross_validation(svc_1 ,features_ren_train,target_ren_train,5)

[ 0.57333333  0.58666667  0.64        0.60666667  0.6       ]
Mean score: 0.601 (+/-0.011)


In [52]:
from sklearn import metrics
    
def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    
    print("Accuaracy on training set:")
    print(clf.score(X_train, y_train))
    
    y_pred = clf.predict(X_test)
    print("Classification Report:")
    print(metrics.classification_report(y_test, y_pred))
    print("Confusion Matrix")
    print(metrics.confusion_matrix(y_test, y_pred))

In [53]:
#the overall model accuracy
train_and_evaluate(svc_1,features_ren_train, features_ren_test, target_ren_train, target_ren_test)

Accuaracy on training set:
0.609333333333
Classification Report:
             precision    recall  f1-score   support

        0.0       0.52      0.71      0.60       110
        1.0       0.68      0.49      0.57       140

avg / total       0.61      0.59      0.59       250

Confusion Matrix
[[78 32]
 [71 69]]


In [54]:
y_pred = svc_1.predict(features_ren_test)

In [55]:
from sklearn import metrics

In [56]:
 print(metrics.classification_report(target_ren_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.52      0.71      0.60       110
        1.0       0.68      0.49      0.57       140

avg / total       0.61      0.59      0.59       250



In [57]:
print(metrics.confusion_matrix(target_ren_test, y_pred))

[[78 32]
 [71 69]]


# SUPPORT VECTOR MACHINE

### Polynomial Kernel of Degree 9

In [58]:
# import the SVM class
from sklearn.svm import SVC
svc_poly = SVC(kernel = 'poly', degree=9) #check regularization parameter

In [64]:
#show results of cross validation
evaluate_cross_validation(svc_poly ,features_ren_train,target_ren_train,5)

[ 0.47333333  0.54        0.55333333  0.56        0.48666667]
Mean score: 0.523 (+/-0.018)


In [59]:
#the overall model accuracy
train_and_evaluate(svc_poly,features_ren_train, features_ren_test, target_ren_train, target_ren_test)

Accuaracy on training set:
0.701333333333
Classification Report:
             precision    recall  f1-score   support

        0.0       0.47      0.85      0.60       110
        1.0       0.67      0.23      0.34       140

avg / total       0.58      0.50      0.46       250

Confusion Matrix
[[ 94  16]
 [108  32]]
